# Fine-tune and Evaluate Amazon Nova Micro model provided by Amazon Bedrock: End-to-End

In this notebook we demonstrate using Boto3 sdk for the fine-tuning and provisioning of [Nova Micro](https://aws.amazon.com/bedrock/nova/) model in Bedrock. You can also do this through the Bedrock Console.


### A Summarization Use Case

In this notebook, we build an end-to-end workflow for fine-tuning and evaluating the Foundation Models (FMs) in Amazon Bedrock. We choose [Amazon Nova Micro](https://aws.amazon.com/bedrock/nova/) as our FM to perform the customization through fine-tuning, we then create on demand inference deployment of the fine-tuned model, and test the on demand invocation using various API methods.

> *This notebook should work well with the **`Data Science 3.0`**, **`Python 3`**, and **`ml.c5.2xlarge`** kernel in SageMaker Studio*

## Prerequisites

 - Make sure you have executed `01_setup_nova_micro.ipynb` notebook.
 - Make sure you are using the same kernel and instance as `01_setup_nova_micro.ipynb` notebook.

In this notebook we demonstrate using Boto3 sdk for the fine-tuning and provisioning of [Nova Micro](https://aws.amazon.com/bedrock/nova/) model in Bedrock. You can also do this through the Bedrock Console.



## Setup

Install and import all the needed libraries and dependencies to complete this notebook.

Please ignore error messages related to pip's dependency resolver.

In [ ]:
# # install the fmeval package for foundation model evaluation

!rm -Rf ~/.cache/pip/*
!pip install tokenizers==0.12.1
!pip install -qU fmeval==0.3.0
!pip install awscurl

#### Setup Tips:

⚠️ ⚠️ ⚠️ If you have trouble installing fmeval, please make sure you have the dependencies installed correctly. See full list of dependencies [here](https://github.com/aws/fmeval/blob/main/poetry.lock). ⚠️ ⚠️ ⚠️ 

In [ ]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Fetching variables from `01_setup_nova_micro.ipynb` notebook. 

In [ ]:
%store -r role_arn
%store -r s3_train_uri
%store -r s3_validation_uri
%store -r s3_test_uri
%store -r bucket_name

In [ ]:
import pprint
pprint.pp(role_arn)
pprint.pp(s3_train_uri)
pprint.pp(s3_validation_uri)
pprint.pp(s3_test_uri)
pprint.pp(bucket_name)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import json
import os
import sys
import boto3
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
session = boto3.session.Session()
region = "us-east-1" # Hardcoded to us-west-2 region
sts_client = boto3.client('sts')
s3_client = boto3.client('s3')
aws_account_id = sts_client.get_caller_identity()["Account"]
bedrock = boto3.client(service_name="bedrock", region_name=region)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region)

In [ ]:
test_file_name = "test-cnn-10.jsonl"
data_folder = "fine-tuning-datasets-nova"

## Create the Fine-Tuning Job

<div class="alert alert-block alert-info">

<b>Note:</b> Fine-tuning job will take around 1-2 hrs to complete</div>

Amazon Nova Micro customization hyperparameters: 

- `epochs`: The number of iterations through the entire training dataset and can take up any integer values in the range of 1-10, with a default value of 2.

- `batchSize`: The number of samples processed before updating model parametersand can take up any integer values in the range of 1-64, with a default value of 1.

- `learningRate`:	The rate at which model parameters are updated after each batch	which can take up a float value betweek 0.0-1.0 with a default value set to	1.00E-5.


For guidelines on setting hyper-parameters refer to the guidelines provided [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-guidelines.html)

In [ ]:
from datetime import datetime
ts = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Choose the foundation model you want to customize and provide ModelId(find more about model reference at https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-reference.html)
base_model_id = "amazon.nova-micro-v1:0:128k"

# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
customization_type = "FINE_TUNING"

# Specify the roleArn for your customization job
customization_role = role_arn

# Create a customization job name
customization_job_name = f"amazon-nova-micro-custom-job-{ts}"

# Create a customized model name for your fine-tuned Nova Micro model
custom_model_name = f"amazon-nova-micro-custom-model-{ts}"

# Define the hyperparameters for fine-tuning Nova Micro model
hyper_parameters = {
        "epochCount": "2",
        "batchSize": "1",
        "learningRate": "0.00005",
    }

# Specify your data path for training, validation(optional) and output
training_data_config = {"s3Uri": s3_train_uri}

# # uncomment the below section if you have validation dataset and provide the s3 uri for it. 

validation_data_config = {
        "validators": [{
            "s3Uri": s3_validation_uri
        }]
    }

output_data_config = {"s3Uri": f's3://{bucket_name}/outputs/output-{custom_model_name}'}

# # Create the customization job
bedrock.create_model_customization_job(
    customizationType=customization_type,
    jobName=customization_job_name,
    customModelName=custom_model_name,
    roleArn=customization_role,
    baseModelIdentifier=base_model_id,
    hyperParameters=hyper_parameters,
    trainingDataConfig=training_data_config,
    validationDataConfig=validation_data_config,
    outputDataConfig=output_data_config

)

## Check Customization Job Status

In [ ]:
import time
fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)["status"]
print(fine_tune_job)

while fine_tune_job == "InProgress":
    time.sleep(60)
    fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)["status"]
    print (fine_tune_job)

## Retrieve Custom Model

Once the customization job is finished, you can check your existing custom model(s) and retrieve the modelArn of your fine-tuned Nova Micro model.

In [ ]:
# You can list your custom models using the command below
bedrock.list_custom_models()

<div class="alert alert-block alert-info">

<b>Note:</b> Please make sure your customization job status is "completed" before proceeding to retrieve the modelArn, otherwise you will run into errors. </div>

In [ ]:
# retrieve the modelArn of the fine-tuned model
fine_tune_job = bedrock.get_custom_model(modelIdentifier=custom_model_name)
custom_model_id = fine_tune_job['modelArn']

In [ ]:
output_job_name = "model-customization-job-"+fine_tune_job['jobArn'].split('/')[-1]
output_job_name

## Visualize Training and Validation Loss

Now that we have completed fine-tuning job, lets visualize our results to see if our job is not underfitting or overfitting. 
Download model customization job metrics from S3 and plot the learning curves.

In [ ]:
output_metrics_path = f"fine-tuning-datasets/{output_job_name}"

In [ ]:
!mkdir $output_metrics_path

In [ ]:
train_metrics_s3_prefix=f'outputs/output-{custom_model_name}/{output_job_name}/training_artifacts/step_wise_training_metrics.csv'
validation_metrics_s3_prefix=f'outputs/output-{custom_model_name}/{output_job_name}/validation_artifacts/post_fine_tuning_validation/validation/validation_metrics.csv'
train_metrics_name='train_metrics.csv'
validation_metrics_name='validation_metrics.csv'
train_file_name_local=output_metrics_path+'/'+train_metrics_name
validation_file_name_local=output_metrics_path+'/'+validation_metrics_name

In [ ]:
s3_client.download_file(bucket_name, train_metrics_s3_prefix, train_file_name_local)
s3_client.download_file(bucket_name, validation_metrics_s3_prefix, validation_file_name_local)

In [ ]:
train_data = pd.read_csv(train_file_name_local)
'''The training loss is at an iteration level. To calculate loss at the epoch level,

    average the iteration-level loss for each epoch'''
train_metrics_epoch=train_data.groupby('step_number').mean()
validation_metrics_epoch=pd.read_csv(validation_file_name_local)
plt.plot(validation_metrics_epoch.epoch_number, validation_metrics_epoch.validation_loss,label='validation')
plt.plot(train_metrics_epoch.index, train_metrics_epoch.training_loss,label='training')
plt.title('Training vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Step')
plt.legend()
plt.show()

## Deploy the Fine-tuned Model

After fine-tuning is complete, we need to deploy the model to make it available for inference. The following command creates a deployment for our fine-tuned Nova Micro model.

In [ ]:
# Create custom model deployment using boto3
response = bedrock.create_custom_model_deployment(
    modelDeploymentName="novafinetunedtestfinal",
    modelArn=custom_model_id,
    description="on-demand custom model deployment",
    tags=[
        {
            'key': 'key',
            'value': 'value'
        }
    ]
)

# Extract the deployment ARN from the response
deployed_model_id = response['customModelDeploymentArn']


# Get Custom Model Deployment Status

### Overview

The `get_custom_model_deployment` API allows you to retrieve detailed information about a custom model deployment in Amazon Bedrock, including its current status, configuration, and metadata.

### API Reference

**Method**: `bedrock.get_custom_model_deployment()`

**Parameters**:
- `customModelDeploymentIdentifier` (string, required): The ARN or name of the custom model deployment

### Response Fields

| Field | Type | Description |
|-------|------|-------------|
| `status` | String | Current deployment status (`Creating`, `Active`, `Failed`) |
| `modelDeploymentName` | String | Name of the custom model deployment |
| `modelArn` | String | ARN of the custom model associated with this deployment |
| `customModelDeploymentArn` | String | ARN of the custom model deployment |
| `description` | String | Description of the deployment |
| `createdAt` | Timestamp | When the deployment was created |
| `lastUpdatedAt` | Timestamp | When the deployment was last updated |
| `failureMessage` | String | Error message if status is `Failed` (optional) |

### Deployment Status Values

- **`Creating`**: The deployment is being set up and prepared for inference
- **`Active`**: The deployment is ready and available for inference requests
- **`Failed`**: The deployment failed to be created or became unavailable



In [ ]:
# Get custom model deployment details
def get_deployment_status(deployment_arn):
    
    try:
        response = bedrock.get_custom_model_deployment(
            customModelDeploymentIdentifier=deployment_arn
        )
        
        return {
            'status': response['status'],
            'deployment_name': response['modelDeploymentName'],
            'model_arn': response['modelArn'],
            'deployment_arn': response['customModelDeploymentArn'],
            'description': response.get('description', ''),
            'created_at': response['createdAt'],
            'last_updated_at': response['lastUpdatedAt'],
            'failure_message': response.get('failureMessage', None)
        }
    except Exception as e:
        print(f"Error retrieving deployment status: {e}")
        return None

deployment_info = get_deployment_status(deployed_model_id)
if deployment_info:
    print(f"Deployment Status: {deployment_info['status']}")
    print(f"Deployment Name: {deployment_info['deployment_name']}")
    print(f"Created At: {deployment_info['created_at']}")
    
    if deployment_info['status'] == 'FAILED' and deployment_info['failure_message']:
        print(f"Failure Reason: {deployment_info['failure_message']}")


## List Custom Model Deployments

### Overview

The `list_custom_model_deployments` API allows you to retrieve a list of all custom model deployments in your AWS account. This is useful for monitoring your deployments, checking their status, and managing your resources.

### Purpose

Use this functionality to:
- **Monitor deployments**: Check the status of all your custom model deployments
- **Resource management**: Get an overview of active deployments and associated costs
- **Troubleshooting**: Identify failed deployments that need attention
- **Inventory management**: Keep track of all deployed custom models

### API Information

**Method**: `bedrock.list_custom_model_deployments()`

**Returns**: A list of deployment summaries with key information about each deployment

### Response Fields

Each deployment summary includes:

| Field | Description |
|-------|-------------|
| `customModelDeploymentName` | The name of the deployment |
| `customModelDeploymentArn` | The unique ARN identifier for the deployment |
| `status` | Current status (`Creating`, `Active`, `Failed`) |
| `modelArn` | ARN of the associated custom model |
| `createdAt` | When the deployment was created |
| `lastUpdatedAt` | When the deployment was last modified |
| `failureMessage` | Error details if status is `Failed` |

### Deployment Status Values

- **`Creating`**: Deployment is being set up
- **`Active`**: Deployment is ready for inference requests
- **`Failed`**: Deployment encountered an error

### Use Cases

1. **Before creating a new deployment**: Check existing deployments to avoid duplicates
2. **Cost monitoring**: Identify active deployments that are incurring charges
3. **Status checking**: Verify deployment readiness before running inference
4. **Cleanup planning**: Find deployments that can be deleted to reduce costs

### Best Practices

- Run this command regularly to monitor your deployment inventory
- Check for `Failed` deployments that may need attention
- Use the output to identify deployments for cleanup
- Keep track of `Active` deployments to understand ongoing costs


In [ ]:
# List all custom model deployments

response = bedrock.list_custom_model_deployments()

deployments = response.get('modelDeploymentSummaries', [])

if deployments:
    print(f"Found {len(deployments)} custom model deployment(s):")
    print("-" * 80)
    
    for deployment in deployments:
        status_emoji = {
            'Active': '🟢',
            'Creating': '🟡',
            'Failed': '🔴'
        }.get(deployment['status'], '⚪')
        
        print(f"{status_emoji} Name: {deployment['customModelDeploymentName']}")
        print(f"   Status: {deployment['status']}")
        print(f"   ARN: {deployment['customModelDeploymentArn']}")
        print(f"   Model ARN: {deployment['modelArn']}")
        print(f"   Created: {deployment['createdAt']}")
        print(f"   Last Updated: {deployment['lastUpdatedAt']}")
        
        # Show failure message if deployment failed
        if deployment.get('failureMessage'):
            print(f"   ❌ Failure: {deployment['failureMessage']}")
        
        print("-" * 80)
    
    # Summary by status
    status_counts = {}
    for deployment in deployments:
        status = deployment['status']
        status_counts[status] = status_counts.get(status, 0) + 1
    
    print(f"\n📊 Status Summary:")
    for status, count in status_counts.items():
        emoji = {'Active': '🟢', 'Creating': '🟡', 'Failed': '🔴'}.get(status, '⚪')
        print(f"   {emoji} {status}: {count}")
        
else:
    print("No custom model deployments found")
        


## Testing the Deployed Model

Now that we have deployed our fine-tuned Nova Micro model, we can test it using various API methods provided by Amazon Bedrock. Each method offers different capabilities for interacting with the model.

### Converse API
The Converse API allows for synchronous conversation with the model, receiving the complete response at once.

In [ ]:
import json

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1')

try:
    response = brt.converse(
        modelId=deployed_model_id,
        system = [
                    {
                      "text": "You are a helpful assistant that summarizes news articles accurately and concisely."
                    }
              ],
        messages=[
                    {
              "role": "user",
              "content": [
                {
                  "text": "Summarize the following news article:\n\n(CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. \"Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fans have all been hurt by your actions,\" NFL Commissioner Roger Goodell said in a letter to Vick. Goodell said he would review the status of the suspension after the legal proceedings are over. In papers filed Friday with a federal court in Virginia, Vick also admitted that he and two co-conspirators killed dogs that did not fight well. Falcons owner Arthur Blank said Vick's admissions describe actions that are \"incomprehensible and unacceptable.\" The suspension makes \"a strong statement that conduct which tarnishes the good reputation of the NFL will not be tolerated,\" he said in a statement.  Watch what led to Vick's suspension \u00bb . Goodell said the Falcons could \"assert any claims or remedies\" to recover $22 million of Vick's signing bonus from the 10-year, $130 million contract he signed in 2004, according to The Associated Press. Vick said he would plead guilty to one count of \"Conspiracy to Travel in Interstate Commerce in Aid of Unlawful Activities and to Sponsor a Dog in an Animal Fighting Venture\" in a plea agreement filed at U.S. District Court in Richmond, Virginia. The charge is punishable by up to five years in prison, a $250,000 fine, \"full restitution, a special assessment and 3 years of supervised release,\" the plea deal said. Federal prosecutors agreed to ask for the low end of the sentencing guidelines. \"The defendant will plead guilty because the defendant is in fact guilty of the charged offense,\" the plea agreement said. In an additional summary of facts, signed by Vick and filed with the agreement, Vick admitted buying pit bulls and the property used for training and fighting the dogs, but the statement said he did not bet on the fights or receive any of the money won. \"Most of the 'Bad Newz Kennels' operations and gambling monies were provided by Vick,\" the official summary of facts said. Gambling wins were generally split among co-conspirators Tony Taylor, Quanis Phillips and sometimes Purnell Peace, it continued. \"Vick did not gamble by placing side bets on any of the fights. Vick did not receive any of the proceeds from the purses that were won by 'Bad Newz Kennels.' \" Vick also agreed that \"collective efforts\" by him and two others caused the deaths of at least six dogs. Around April, Vick, Peace and Phillips tested some dogs in fighting sessions at Vick's property in Virginia, the statement said. \"Peace, Phillips and Vick agreed to the killing of approximately 6-8 dogs that did not perform well in 'testing' sessions at 1915 Moonlight Road and all of those dogs were killed by various methods, including hanging and drowning. \"Vick agrees and stipulates that these dogs all died as a result of the collective efforts of Peace, Phillips and Vick,\" the summary said. Peace, 35, of Virginia Beach, Virginia; Phillips, 28, of Atlanta, Georgia; and Taylor, 34, of Hampton, Virginia, already have accepted agreements to plead guilty in exchange for reduced sentences. Vick, 27, is scheduled to appear Monday in court, where he is expected to plead guilty before a judge.  See a timeline of the case against Vick \u00bb . The judge in the case will have the final say over the plea agreement. The federal case against Vick focused on the interstate conspiracy, but Vick's admission that he was involved in the killing of dogs could lead to local charges, according to CNN legal analyst Jeffrey Toobin. \"It sometimes happens -- not often -- that the state will follow a federal prosecution by charging its own crimes for exactly the same behavior,\" Toobin said Friday. \"The risk for Vick is, if he makes admissions in his federal guilty plea, the state of Virginia could say, 'Hey, look, you admitted violating Virginia state law as well. We're going to introduce that against you and charge you in our court.' \" In the plea deal, Vick agreed to cooperate with investigators and provide all information he may have on any criminal activity and to testify if necessary. Vick also agreed to turn over any documents he has and to submit to polygraph tests. Vick agreed to \"make restitution for the full amount of the costs associated\" with the dogs that are being held by the government. \"Such costs may include, but are not limited to, all costs associated with the care of the dogs involved in that case, including if necessary, the long-term care and/or the humane euthanasia of some or all of those animals.\" Prosecutors, with the support of animal rights activists, have asked for permission to euthanize the dogs. But the dogs could serve as important evidence in the cases against Vick and his admitted co-conspirators. Judge Henry E. Hudson issued an order Thursday telling the U.S. Marshals Service to \"arrest and seize the defendant property, and use discretion and whatever means appropriate to protect and maintain said defendant property.\" Both the judge's order and Vick's filing refer to \"approximately\" 53 pit bull dogs. After Vick's indictment last month, Goodell ordered the quarterback not to report to the Falcons training camp, and the league is reviewing the case. Blank told the NFL Network on Monday he could not speculate on Vick's future as a Falcon, at least not until he had seen \"a statement of facts\" in the case.  E-mail to a friend . CNN's Mike Phelan contributed to this report."
                }
              ]
            }
        ]
    )

    print("Request ID:", response['ResponseMetadata']['RequestId'])
    result = response.get('output')
    print(result)
    print("\n\n\n")
    print(result['message']['content'][0]['text'])

except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])      

### Converse Stream API
The Converse Stream API provides the response as a stream of tokens, allowing for faster time-to-first-token and progressive rendering of the response.

In [ ]:
import boto3
import json

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1')

try:
    response = brt.converse_stream(
        
        modelId=deployed_model_id,
        system = [
                    {
                      "text": "You are a helpful assistant that summarizes news articles accurately and concisely."
                    }
              ],
        messages=[
                    {
              "role": "user",
              "content": [
                {
                  "text": "Summarize the following news article:\n\n(CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. \"Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fans have all been hurt by your actions,\" NFL Commissioner Roger Goodell said in a letter to Vick. Goodell said he would review the status of the suspension after the legal proceedings are over. In papers filed Friday with a federal court in Virginia, Vick also admitted that he and two co-conspirators killed dogs that did not fight well. Falcons owner Arthur Blank said Vick's admissions describe actions that are \"incomprehensible and unacceptable.\" The suspension makes \"a strong statement that conduct which tarnishes the good reputation of the NFL will not be tolerated,\" he said in a statement.  Watch what led to Vick's suspension \u00bb . Goodell said the Falcons could \"assert any claims or remedies\" to recover $22 million of Vick's signing bonus from the 10-year, $130 million contract he signed in 2004, according to The Associated Press. Vick said he would plead guilty to one count of \"Conspiracy to Travel in Interstate Commerce in Aid of Unlawful Activities and to Sponsor a Dog in an Animal Fighting Venture\" in a plea agreement filed at U.S. District Court in Richmond, Virginia. The charge is punishable by up to five years in prison, a $250,000 fine, \"full restitution, a special assessment and 3 years of supervised release,\" the plea deal said. Federal prosecutors agreed to ask for the low end of the sentencing guidelines. \"The defendant will plead guilty because the defendant is in fact guilty of the charged offense,\" the plea agreement said. In an additional summary of facts, signed by Vick and filed with the agreement, Vick admitted buying pit bulls and the property used for training and fighting the dogs, but the statement said he did not bet on the fights or receive any of the money won. \"Most of the 'Bad Newz Kennels' operations and gambling monies were provided by Vick,\" the official summary of facts said. Gambling wins were generally split among co-conspirators Tony Taylor, Quanis Phillips and sometimes Purnell Peace, it continued. \"Vick did not gamble by placing side bets on any of the fights. Vick did not receive any of the proceeds from the purses that were won by 'Bad Newz Kennels.' \" Vick also agreed that \"collective efforts\" by him and two others caused the deaths of at least six dogs. Around April, Vick, Peace and Phillips tested some dogs in fighting sessions at Vick's property in Virginia, the statement said. \"Peace, Phillips and Vick agreed to the killing of approximately 6-8 dogs that did not perform well in 'testing' sessions at 1915 Moonlight Road and all of those dogs were killed by various methods, including hanging and drowning. \"Vick agrees and stipulates that these dogs all died as a result of the collective efforts of Peace, Phillips and Vick,\" the summary said. Peace, 35, of Virginia Beach, Virginia; Phillips, 28, of Atlanta, Georgia; and Taylor, 34, of Hampton, Virginia, already have accepted agreements to plead guilty in exchange for reduced sentences. Vick, 27, is scheduled to appear Monday in court, where he is expected to plead guilty before a judge.  See a timeline of the case against Vick \u00bb . The judge in the case will have the final say over the plea agreement. The federal case against Vick focused on the interstate conspiracy, but Vick's admission that he was involved in the killing of dogs could lead to local charges, according to CNN legal analyst Jeffrey Toobin. \"It sometimes happens -- not often -- that the state will follow a federal prosecution by charging its own crimes for exactly the same behavior,\" Toobin said Friday. \"The risk for Vick is, if he makes admissions in his federal guilty plea, the state of Virginia could say, 'Hey, look, you admitted violating Virginia state law as well. We're going to introduce that against you and charge you in our court.' \" In the plea deal, Vick agreed to cooperate with investigators and provide all information he may have on any criminal activity and to testify if necessary. Vick also agreed to turn over any documents he has and to submit to polygraph tests. Vick agreed to \"make restitution for the full amount of the costs associated\" with the dogs that are being held by the government. \"Such costs may include, but are not limited to, all costs associated with the care of the dogs involved in that case, including if necessary, the long-term care and/or the humane euthanasia of some or all of those animals.\" Prosecutors, with the support of animal rights activists, have asked for permission to euthanize the dogs. But the dogs could serve as important evidence in the cases against Vick and his admitted co-conspirators. Judge Henry E. Hudson issued an order Thursday telling the U.S. Marshals Service to \"arrest and seize the defendant property, and use discretion and whatever means appropriate to protect and maintain said defendant property.\" Both the judge's order and Vick's filing refer to \"approximately\" 53 pit bull dogs. After Vick's indictment last month, Goodell ordered the quarterback not to report to the Falcons training camp, and the league is reviewing the case. Blank told the NFL Network on Monday he could not speculate on Vick's future as a Falcon, at least not until he had seen \"a statement of facts\" in the case.  E-mail to a friend . CNN's Mike Phelan contributed to this report."
                }
              ]
            }
        ]
    )

    for chunk in response["stream"]:
        if "contentBlockDelta" in chunk:
            text = chunk["contentBlockDelta"]["delta"]["text"]
            print(text, end="")
    
except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])

### Invoke API
The Invoke API allows for direct model invocation with custom system prompts and inference parameters.

In [ ]:
import boto3
import json

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1')


system_list = [
                    {
                      "text": "You are a helpful assistant that summarizes news articles accurately and concisely."
                    }
              ]
message_list=[
            {
      "role": "user",
      "content": [
        {
          "text": "Summarize the following news article:\n\n(CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. \"Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fans have all been hurt by your actions,\" NFL Commissioner Roger Goodell said in a letter to Vick. Goodell said he would review the status of the suspension after the legal proceedings are over. In papers filed Friday with a federal court in Virginia, Vick also admitted that he and two co-conspirators killed dogs that did not fight well. Falcons owner Arthur Blank said Vick's admissions describe actions that are \"incomprehensible and unacceptable.\" The suspension makes \"a strong statement that conduct which tarnishes the good reputation of the NFL will not be tolerated,\" he said in a statement.  Watch what led to Vick's suspension \u00bb . Goodell said the Falcons could \"assert any claims or remedies\" to recover $22 million of Vick's signing bonus from the 10-year, $130 million contract he signed in 2004, according to The Associated Press. Vick said he would plead guilty to one count of \"Conspiracy to Travel in Interstate Commerce in Aid of Unlawful Activities and to Sponsor a Dog in an Animal Fighting Venture\" in a plea agreement filed at U.S. District Court in Richmond, Virginia. The charge is punishable by up to five years in prison, a $250,000 fine, \"full restitution, a special assessment and 3 years of supervised release,\" the plea deal said. Federal prosecutors agreed to ask for the low end of the sentencing guidelines. \"The defendant will plead guilty because the defendant is in fact guilty of the charged offense,\" the plea agreement said. In an additional summary of facts, signed by Vick and filed with the agreement, Vick admitted buying pit bulls and the property used for training and fighting the dogs, but the statement said he did not bet on the fights or receive any of the money won. \"Most of the 'Bad Newz Kennels' operations and gambling monies were provided by Vick,\" the official summary of facts said. Gambling wins were generally split among co-conspirators Tony Taylor, Quanis Phillips and sometimes Purnell Peace, it continued. \"Vick did not gamble by placing side bets on any of the fights. Vick did not receive any of the proceeds from the purses that were won by 'Bad Newz Kennels.' \" Vick also agreed that \"collective efforts\" by him and two others caused the deaths of at least six dogs. Around April, Vick, Peace and Phillips tested some dogs in fighting sessions at Vick's property in Virginia, the statement said. \"Peace, Phillips and Vick agreed to the killing of approximately 6-8 dogs that did not perform well in 'testing' sessions at 1915 Moonlight Road and all of those dogs were killed by various methods, including hanging and drowning. \"Vick agrees and stipulates that these dogs all died as a result of the collective efforts of Peace, Phillips and Vick,\" the summary said. Peace, 35, of Virginia Beach, Virginia; Phillips, 28, of Atlanta, Georgia; and Taylor, 34, of Hampton, Virginia, already have accepted agreements to plead guilty in exchange for reduced sentences. Vick, 27, is scheduled to appear Monday in court, where he is expected to plead guilty before a judge.  See a timeline of the case against Vick \u00bb . The judge in the case will have the final say over the plea agreement. The federal case against Vick focused on the interstate conspiracy, but Vick's admission that he was involved in the killing of dogs could lead to local charges, according to CNN legal analyst Jeffrey Toobin. \"It sometimes happens -- not often -- that the state will follow a federal prosecution by charging its own crimes for exactly the same behavior,\" Toobin said Friday. \"The risk for Vick is, if he makes admissions in his federal guilty plea, the state of Virginia could say, 'Hey, look, you admitted violating Virginia state law as well. We're going to introduce that against you and charge you in our court.' \" In the plea deal, Vick agreed to cooperate with investigators and provide all information he may have on any criminal activity and to testify if necessary. Vick also agreed to turn over any documents he has and to submit to polygraph tests. Vick agreed to \"make restitution for the full amount of the costs associated\" with the dogs that are being held by the government. \"Such costs may include, but are not limited to, all costs associated with the care of the dogs involved in that case, including if necessary, the long-term care and/or the humane euthanasia of some or all of those animals.\" Prosecutors, with the support of animal rights activists, have asked for permission to euthanize the dogs. But the dogs could serve as important evidence in the cases against Vick and his admitted co-conspirators. Judge Henry E. Hudson issued an order Thursday telling the U.S. Marshals Service to \"arrest and seize the defendant property, and use discretion and whatever means appropriate to protect and maintain said defendant property.\" Both the judge's order and Vick's filing refer to \"approximately\" 53 pit bull dogs. After Vick's indictment last month, Goodell ordered the quarterback not to report to the Falcons training camp, and the league is reviewing the case. Blank told the NFL Network on Monday he could not speculate on Vick's future as a Falcon, at least not until he had seen \"a statement of facts\" in the case.  E-mail to a friend . CNN's Mike Phelan contributed to this report."
        }
      ]
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 500, "topP": 0.9, "topK": 20, "temperature": 0.7}
request_body = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params

}

body = json.dumps(request_body)
try:
    response = brt.invoke_model(
        modelId=deployed_model_id,
        body=body

    )
    print("Request ID:", response['ResponseMetadata']['RequestId'])

    # Decode the response body.
    model_response = json.loads(response["body"].read())
    # Extract and print the response text.
    response_text = model_response["output"]["message"]["content"][0]["text"]
    print(response_text)

except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])


### Invoke Stream API
Similar to the Converse Stream API, this provides streaming responses but with the Invoke API's flexibility for custom configurations.

In [ ]:
import boto3
import json
from datetime import datetime

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1')

system_list = [
                    {
                      "text": "You are a helpful assistant that summarizes news articles accurately and concisely."
                    }
              ]
message_list=[
            {
      "role": "user",
      "content": [
        {
          "text": "Summarize the following news article:\n\n(CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. \"Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fans have all been hurt by your actions,\" NFL Commissioner Roger Goodell said in a letter to Vick. Goodell said he would review the status of the suspension after the legal proceedings are over. In papers filed Friday with a federal court in Virginia, Vick also admitted that he and two co-conspirators killed dogs that did not fight well. Falcons owner Arthur Blank said Vick's admissions describe actions that are \"incomprehensible and unacceptable.\" The suspension makes \"a strong statement that conduct which tarnishes the good reputation of the NFL will not be tolerated,\" he said in a statement.  Watch what led to Vick's suspension \u00bb . Goodell said the Falcons could \"assert any claims or remedies\" to recover $22 million of Vick's signing bonus from the 10-year, $130 million contract he signed in 2004, according to The Associated Press. Vick said he would plead guilty to one count of \"Conspiracy to Travel in Interstate Commerce in Aid of Unlawful Activities and to Sponsor a Dog in an Animal Fighting Venture\" in a plea agreement filed at U.S. District Court in Richmond, Virginia. The charge is punishable by up to five years in prison, a $250,000 fine, \"full restitution, a special assessment and 3 years of supervised release,\" the plea deal said. Federal prosecutors agreed to ask for the low end of the sentencing guidelines. \"The defendant will plead guilty because the defendant is in fact guilty of the charged offense,\" the plea agreement said. In an additional summary of facts, signed by Vick and filed with the agreement, Vick admitted buying pit bulls and the property used for training and fighting the dogs, but the statement said he did not bet on the fights or receive any of the money won. \"Most of the 'Bad Newz Kennels' operations and gambling monies were provided by Vick,\" the official summary of facts said. Gambling wins were generally split among co-conspirators Tony Taylor, Quanis Phillips and sometimes Purnell Peace, it continued. \"Vick did not gamble by placing side bets on any of the fights. Vick did not receive any of the proceeds from the purses that were won by 'Bad Newz Kennels.' \" Vick also agreed that \"collective efforts\" by him and two others caused the deaths of at least six dogs. Around April, Vick, Peace and Phillips tested some dogs in fighting sessions at Vick's property in Virginia, the statement said. \"Peace, Phillips and Vick agreed to the killing of approximately 6-8 dogs that did not perform well in 'testing' sessions at 1915 Moonlight Road and all of those dogs were killed by various methods, including hanging and drowning. \"Vick agrees and stipulates that these dogs all died as a result of the collective efforts of Peace, Phillips and Vick,\" the summary said. Peace, 35, of Virginia Beach, Virginia; Phillips, 28, of Atlanta, Georgia; and Taylor, 34, of Hampton, Virginia, already have accepted agreements to plead guilty in exchange for reduced sentences. Vick, 27, is scheduled to appear Monday in court, where he is expected to plead guilty before a judge.  See a timeline of the case against Vick \u00bb . The judge in the case will have the final say over the plea agreement. The federal case against Vick focused on the interstate conspiracy, but Vick's admission that he was involved in the killing of dogs could lead to local charges, according to CNN legal analyst Jeffrey Toobin. \"It sometimes happens -- not often -- that the state will follow a federal prosecution by charging its own crimes for exactly the same behavior,\" Toobin said Friday. \"The risk for Vick is, if he makes admissions in his federal guilty plea, the state of Virginia could say, 'Hey, look, you admitted violating Virginia state law as well. We're going to introduce that against you and charge you in our court.' \" In the plea deal, Vick agreed to cooperate with investigators and provide all information he may have on any criminal activity and to testify if necessary. Vick also agreed to turn over any documents he has and to submit to polygraph tests. Vick agreed to \"make restitution for the full amount of the costs associated\" with the dogs that are being held by the government. \"Such costs may include, but are not limited to, all costs associated with the care of the dogs involved in that case, including if necessary, the long-term care and/or the humane euthanasia of some or all of those animals.\" Prosecutors, with the support of animal rights activists, have asked for permission to euthanize the dogs. But the dogs could serve as important evidence in the cases against Vick and his admitted co-conspirators. Judge Henry E. Hudson issued an order Thursday telling the U.S. Marshals Service to \"arrest and seize the defendant property, and use discretion and whatever means appropriate to protect and maintain said defendant property.\" Both the judge's order and Vick's filing refer to \"approximately\" 53 pit bull dogs. After Vick's indictment last month, Goodell ordered the quarterback not to report to the Falcons training camp, and the league is reviewing the case. Blank told the NFL Network on Monday he could not speculate on Vick's future as a Falcon, at least not until he had seen \"a statement of facts\" in the case.  E-mail to a friend . CNN's Mike Phelan contributed to this report."
        }
      ]
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 500, "topP": 0.9, "topK": 20, "temperature": 0.7}

request_body = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

try:
    start_time = datetime.now()
    chunk_count = 0
    time_to_first_token = None
    response = brt.invoke_model_with_response_stream(
        modelId=deployed_model_id,
        body=json.dumps(request_body)
    )

    stream = response.get("body")
    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                # Print the response chunk
                chunk_json = json.loads(chunk.get("bytes").decode())
                # Pretty print JSON
                # print(json.dumps(chunk_json, indent=2, ensure_ascii=False))

                content_block_delta = chunk_json.get("contentBlockDelta")
                if content_block_delta:
                    if time_to_first_token is None:
                        time_to_first_token = datetime.now() - start_time
                        print(f"Time to first token: {time_to_first_token}")
    
                    chunk_count += 1
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S:%f")
                    # print(f"{current_time} - ", end="")
                    print(content_block_delta.get("delta").get("text"), end="")

        print(f"Total chunks: {chunk_count}")

    else:
        print("No response stream received.")

except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])

## Cleanup Resources

You can run the below code to delete the custom model deployment



In [ ]:

response = bedrock.delete_custom_model_deployment(
    customModelDeploymentIdentifier=deployed_model_id
)


## Summary

- Installed additional packages (fmeval, tokenizers, awscurl) and retrieves stored variables from setup notebook
- Create and monitor Nova Micro fine-tuning job with custom hyperparameters (2 epochs, batch size 1, learning rate 0.00005) that takes ~60 minutes to complete
- Download training and validation metrics from S3 and visualizes loss curves to evaluate model performance and check for overfitting/underfitting
- Deploy the fine-tuned Nova Micro model as an on-demand inference endpoint using awscurl command for real-time inference capabilities
- Test the on-demand inference deployment using four different API methods: Converse API (synchronous responses), Converse Stream API (streaming tokens), Invoke API (custom parameters), and Invoke Stream API (streaming with custom configurations)